In [1]:
import pandas as pd
import numpy as np
from typing import Any
try:
    from typing import Self
except ImportError:
    from typing_extensions import Self

In [2]:
def array_to_str(arr):
    if arr.ndim == 1:
        return ','.join(map(str, np.around(arr, 4).tolist()))
    elif arr.ndim == 2:
        return '\n'.join((','.join(map(str, x)) for x in np.around(arr, 4).tolist()))
    else:
        raise ValueError('Only 1 and 2-d arrays suppported')

def write(file: str, data: Any):
    with open(file, 'w') as f:
        f.write(data if isinstance(data, str) else str(data))

def rescale(x: np.ndarray) -> np.ndarray:
    min_ = np.min(x)
    range_ = np.ptp(x)
    return (x - min_) / range_

In [3]:
df = pd.read_csv('time_series_covid19_deaths_US_raw.csv')
raw_data = df.to_numpy()

In [4]:
cum_data = np.cumsum(raw_data, axis=0)
data = np.diff(cum_data)

del raw_data

In [5]:
# wisc_cal_data = cum_data[(4, 48), :]
# q1_str = array_to_str(wisc_cal_data)
# write('q1.txt', q1_str)

# del cum_data
# del q1_str

In [6]:
# q2_data = data[(4, 48), :] # 4-> cal, 48 -> wisco
# q2_str = array_to_str(q2_data)
# write('q2.txt', q2_str)

# del q2_data
# del q2_str

In [7]:
# write('q3.txt', '''Mean of the time-differenced data
# Standard deviation of the time-differenced data
# Median of the time-differenced data
# Linear trend coefficient of the data
# Auto-correlation of the data''')

In [8]:
mean = np.mean(data, axis=1)
mean = rescale(mean)

In [9]:
std = np.std(data, axis=1)
std = rescale(std)

In [10]:
median = np.median(data, axis=1)
median = rescale(median)

In [11]:
ltc = np.array([sum((x[i] - mean[idx]) * (i + 1 - (half_length := ((data.shape[-1] + 1) / 2))) for i in range(len(x))) for idx, x in enumerate(data.tolist())]) / sum((i+1 - half_length) ** 2 for i in range(data.shape[-1]))
ltc = rescale(ltc)

In [12]:
ac = np.array([ sum((x[i] * x[i-1] for i in range(1, len(x)))) / np.sum(np.power(x, 2)) for x in (data - mean.reshape(50, -1)).tolist()])
ac = rescale(ac)

In [13]:
parametric_data = np.empty((50, 5))
parametric_data[:, 0] = mean
parametric_data[:, 1] = std
parametric_data[:, 2] = median
parametric_data[:, 3] = ltc
parametric_data[:, 4] = ac

# q4_str = array_to_str(parametric_data)
# write('q4.txt', q4_str)
# del q4_str

In [14]:
class HierarchicalClustering:
    class Clustering:
        @staticmethod
        def _compute_distances(X: np.ndarray) -> np.ndarray:
            n = X.shape[0]
            distance_matrix = np.zeros((n, n))
        
            for i in range(n):
                for j in range(i + 1, n):
                    distance_matrix[i, j] = np.linalg.norm(X[i] - X[j])
        
            distance_matrix += distance_matrix.T
        
            return distance_matrix
        
        @staticmethod
        def _single(distance_matrix: np.ndarray, num_clusters: int) -> list:
            num_points = distance_matrix.shape[0]
            clusters = [[i] for i in range(num_points)]
        
            while len(clusters) > num_clusters:
                min_distance = np.inf
                merge_index1, merge_index2 = None, None
        
                for i in range(len(clusters)):
                    for j in range(i + 1, len(clusters)):
                        cluster1 = clusters[i]
                        cluster2 = clusters[j]
        
                        distance = np.min(distance_matrix[np.ix_(cluster1, cluster2)])
        
                        if distance < min_distance:
                            min_distance = distance
                            merge_index1, merge_index2 = i, j
        
                # Merge the two closest clusters
                clusters[merge_index1].extend(clusters[merge_index2])
                del clusters[merge_index2]
        
                combined_cluster = clusters[merge_index1]
                for i in range(distance_matrix.shape[0]):
                    if i in combined_cluster:
                        continue
                    distance = np.min(distance_matrix[np.ix_(combined_cluster, [i])])
                    distance_matrix[i, combined_cluster] = distance
                    distance_matrix[combined_cluster, i] = distance
        
            return tuple(map(tuple, clusters))
    
        @staticmethod
        def _complete(distance_matrix: np.ndarray, n_clusters: int) -> list:
            num_points = distance_matrix.shape[0]
            clusters = [[i] for i in range(num_points)]
        
            while len(clusters) > n_clusters:
                min_max_distance = np.inf
                merge_index1, merge_index2 = None, None
        
                for i in range(len(clusters)):
                    for j in range(i + 1, len(clusters)):
                        cluster1 = clusters[i]
                        cluster2 = clusters[j]
        
                        max_distance = np.max(distance_matrix[np.ix_(cluster1, cluster2)])
        
                        if max_distance < min_max_distance:
                            min_max_distance = max_distance
                            merge_index1, merge_index2 = i, j
        
                clusters[merge_index1].extend(clusters[merge_index2])
                del clusters[merge_index2]
        
                combined_cluster = clusters[merge_index1]
                for i in range(distance_matrix.shape[0]):
                    if i in combined_cluster:
                        continue
                    distance = np.max(distance_matrix[np.ix_(combined_cluster, [i])])
                    distance_matrix[i, combined_cluster] = distance
                    distance_matrix[combined_cluster, i] = distance
        
            return clusters
        
    _METHODS = {
        'single': Clustering._single,
        'complete': Clustering._complete,
    }

    def __init__(self, n_clusters: int = 2, linkage: str = 'single'):
        self._n_clusters = n_clusters
        
        assert HierarchicalClustering._is_linkage_valid(linkage), (
            f'Invalid Linkage, valid linkages are \'single\' and '
            f'\'complete\', found {linkage!r}'
        )
        
        self._linkage = linkage

    def _build(self, X: np.ndarray):
        distance_matrix = HierarchicalClustering.Clustering._compute_distances(X)
        cluster_func = HierarchicalClustering._METHODS.get(self._linkage)
        if cluster_func is None:
            raise ValueError(
                f'Invalid Linkage, the Hierarchical Clustering instance '
                f'may have been tampered with. found {self.linkage!r}'
            )

        self._clusters = cluster_func(distance_matrix, self._n_clusters)

    def fit(self, X: np.ndarray) -> Self:
        self._build(X)
        return self

    def predict(self, X: np.ndarray):
        pass
    
    @property
    def clusters(self):
        return self._clusters

    @clusters.setter
    def clusters(self, value):
        raise ValueError('The `clusters` attribute should not be modified externally!')
    
    @staticmethod
    def _is_linkage_valid(linkage: str) -> bool:
        return linkage in HierarchicalClustering._METHODS.keys()

In [15]:
slhc = HierarchicalClustering(n_clusters = 5)
slhc = slhc.fit(parametric_data)

idxs = np.arange(parametric_data.shape[0])

for cluster_idx, cluster in enumerate(slhc.clusters):
    for item in cluster:
        idxs[item] = cluster_idx

# q5_str = array_to_str(idxs)
# write('q5.txt', q5_str)
# del q5_str

In [16]:
clhc = HierarchicalClustering(n_clusters = 5, linkage='complete')
clhc = clhc.fit(parametric_data)

idxs = np.arange(parametric_data.shape[0])

for cluster_idx, cluster in enumerate(clhc.clusters):
    for item in cluster:
        idxs[item] = cluster_idx

# q6_str = array_to_str(idxs)
# write('q6.txt', q6_str)
# del q6_str

In [17]:
class KMeans:
    class Clustering:
        @staticmethod
        def assign_points(centroids: np.ndarray, X: np.ndarray) -> np.ndarray:
            distances = KMeans.Clustering.compute_distances(centroids, X)

            return np.argmin(distances, axis=1)
        
        @staticmethod
        def choose_centroids(X: np.ndarray, k: int = 2) -> np.ndarray:
            return np.random.default_rng().choice(
                X,
                size=k,
                replace=False,
                shuffle=False
            )

        @staticmethod
        def compute_centroids(X: np.ndarray, assignments: np.ndarray, k: int) -> np.ndarray:
            centroids = np.empty((k, X.shape[1]))
            for i in range(k):
                idxs = assignments == i
                centroids[i] = np.mean(X[idxs], axis=0)
            return centroids
        
        @staticmethod
        def compute_distances(centroids: np.ndarray, X: np.ndarray) -> np.ndarray:
            return np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
    
    def __init__(self, k: int = 2, max_iters: int = 100, tol: float = 1e-5, ):
        self._k = k
        self._max_iters = max_iters
        self._tol = abs(tol)
    
    def _build(self, X: np.ndarray):
        self._centroids = KMeans.Clustering.choose_centroids(X, self.k)

        old_centroids = np.empty_like(self._centroids)
        for _ in range(self._max_iters):
            np.copyto(old_centroids, self._centroids)
            
            assignments = KMeans.Clustering.assign_points(self._centroids, X)

            self._centroids = KMeans.Clustering.compute_centroids(X, assignments, self.k)

            if np.all(np.abs(self.centroids - old_centroids) < self._tol):
                self._converged_in = _
                break
    
    def fit(self, X: np.ndarray) -> Self:
        self._build(X)
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return KMeans.Clustering.assign_points(self._centroids, X)

    def distortion(self, X: np.ndarray) -> np.ndarray:
        assignments = self.predict(X)
        _distortion = 0
        
        for i in range(centroids.shape[0]):
            idxs = assignments == i
            _distortion += np.sum((X[idxs] - self._centroids[i]) ** 2)
    
        return _distortion
    
    @property
    def centroids(self):
        return self._centroids
    
    @centroids.setter
    def centroids(self):
        raise ValueError('The `centroids` attribute should not be modified externally!')
    
    @property
    def k(self):
        return self._k
    
    @k.setter
    def k(self):
        raise ValueError('The `k` attribute should not be modified externally!')

In [18]:
kmeans = KMeans(k=5)
kmeans.fit(parametric_data)

In [19]:
predictions = kmeans.predict(parametric_data)
print(predictions)

q7_str = array_to_str(predictions)
write('q7.txt', q7_str)

[4 4 4 4 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3
 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [20]:
centroids = kmeans.centroids
print(centroids)

q8_str = array_to_str(centroids)
write('q8.txt', q8_str)

[[0.47093786 0.47923711 0.49213724 0.58130065 0.82953252]
 [0.24875333 0.27621173 0.2607057  0.77092614 0.73894416]
 [0.90942921 0.91098634 0.91746852 0.05107117 0.98803245]
 [0.67874628 0.67784625 0.68977841 0.25297739 0.9349062 ]
 [0.01910688 0.0263451  0.01608292 0.98534334 0.09319708]]


In [21]:
distortion = kmeans.distortion(parametric_data)
print(distortion)

q9_str = str(np.around(distortion, 4))
write('q9.txt', q9_str)

1.0194499329842523


In [22]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init='auto')
kmeans.fit(parametric_data)

print('kmeans.labels_', kmeans.labels_, sep='\n')
print('kmeans.cluster_centers_', kmeans.cluster_centers_, sep='\n')
print('kmeans.inertia_', kmeans.inertia_, sep='\n')

kmeans.labels_
[2 2 2 2 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
kmeans.cluster_centers_
[[0.3841545  0.4013964  0.40084187 0.65194206 0.78788545]
 [0.85667248 0.85702788 0.86474466 0.09064934 0.97840011]
 [0.01910688 0.0263451  0.01608292 0.98534334 0.09319708]
 [0.54569964 0.54870142 0.56709338 0.48836738 0.86210152]
 [0.21272193 0.24230699 0.22399929 0.80436664 0.73502808]]
kmeans.inertia_
1.2250582936983982


In [23]:
round(kmeans.inertia_, 4)

1.2251